In [2]:
import cv2

cap = cv2.VideoCapture("../images/woman_who_is_running.avi")

ret, frame = cap.read()
cv2.imshow("selectROI", frame) #ROI 선택을 위해 영상 첫번째 프레임을 불러와 출력
x, y, w, h = cv2.selectROI("selectROI", frame) #ROI선택함수 실행, ROI 선택 인터페이스 출력

rc = (x, y, w, h) #선택한 ROI영역의 좌표정보를 튜플로 rc 변수에 저장
roi = frame[y:y+h, x:x+w] #선택한 ROI영역을 frame에서 크롭(crop)한 후 roi변수에 저장
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) #색상 히스토그램 생성을 위해 크롭한 영역을 HSV색공간으로 변환
channels = [0, 1] #히스토그램 생성시 사용할 색정보 채널을 H, S로 선택
ranges = [0, 180, 0, 256] #H의 범위는 0~179, S의 범위는 0~255로 설정

hist = cv2.calcHist([roi_hsv], channels, None, [200, 200], ranges) #색상 히스토그램 생성하여 hist 변수에 저장

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1) #mean shift 알고리즘 종료기준 설정
while True: #동영상 실시간 처리
    ret, frame = cap.read()
    if ret is False:
        break
    #불러온 frame을 hist를 이용, 역투영하기 위해 HSV색공간으로 변환
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #frame에 hist를 역투영하여 부합하는 색정보만 남기고 제거, backproj 변수에 저장
    backproj = cv2.calcBackProject([hsv], channels, hist, ranges, 1)
    _, rc = cv2.meanShift(backproj, rc, term_crit) #backproj에 mean shift 실행
    cv2.rectangle(frame, rc, (0, 0, 255), 2) #mean shift된 window 좌표에 사각형 생성
    cv2.imshow("frame", frame)
    if cv2.waitKey(25) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
